python 3.8 + pytorch 1.8

# 搭建Vanilla RNN

- 目的：搭建简单的RNN模型并进行模拟句子分类
- 主要步骤：
1. 输入尺寸分析
2. 构建模型
3. 模型初始化
4 .测试

##  1. 输入尺寸分析

- 推荐阅读材料：
[LSTM pytorch参数理解](https://zhuanlan.zhihu.com/p/340763181)-知乎

In [12]:
import torch
import torch.nn as nn
from torchstat import stat
from torchsummary import summary
# input_size 为输入数据维数，即单词稀疏向量长度
# hidden_size 为输出维数，例如有30个分类
lstm = nn.LSTM(input_size=10, hidden_size=30, num_layers=1)

seq_len = 2 # 表示参与训练的序列长度（句子长度）
batch_size = 5 # batch_size代表一批几个序列参与训练（几个句子参与训练）
input_size = 10 # 单词稀疏向量长度
# 模拟数据
x = torch.randn((seq_len, batch_size, input_size))
print(x)
y, (h, c) = lstm(x)
print(x.shape)
print(y.shape)
print(h.shape) # 单向单层LSTM隐藏状态0维为1
print(c.shape) # 同h
# stat(lstm, input_size=(2,10,1))
# summary(lstm.cuda(), input_size=(2,10,), batch_size=10)

tensor([[[-0.7984,  2.9845, -0.0627, -2.5412, -1.4661, -1.1345,  1.2943,
           1.0295,  0.3614,  0.9838],
         [-2.2107,  1.1269, -0.1324,  1.5966, -0.2545,  0.1964,  0.5977,
           0.0629,  1.6581, -1.8710],
         [ 0.4070,  1.2150, -0.1763, -0.6439, -0.6503,  0.0745, -0.2769,
           0.0507,  2.1819, -1.5359],
         [ 0.4931,  0.4234, -2.0911, -0.3107, -0.8948,  0.2314,  0.6304,
          -1.0792,  0.1387, -2.1024],
         [-2.7084,  1.9043,  1.0090,  2.0205,  0.6874,  0.6515, -0.5708,
           0.2782, -2.0245,  1.5090]],

        [[-0.2331,  0.8968,  0.1058, -0.8904,  0.9551, -0.3846,  1.1521,
           0.3353, -0.0336,  1.8635],
         [-0.5744, -0.1069,  0.4371, -1.8343,  0.4586, -0.2221, -1.0703,
           1.4535, -0.0518,  1.4833],
         [-0.2980,  0.7146,  0.0996, -1.2401, -1.0239,  0.8318, -0.5858,
          -0.7593,  0.7798, -0.5904],
         [-0.0862,  0.5282,  1.2474,  1.0868,  0.5100,  1.4611, -0.3139,
          -1.0567,  1.4279, -0.4733],

## 2. 构建模型：

In [67]:
# 结构：输入层->embed层->lstm层->fc层
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)  # 词向量层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)  # LSTM 层
        # 全连接输出分类层, output_size 为类别大小，输出各词概率
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        # x: [seq_len, batch_size]
        embeds = self.embed(x)  # 经由词向量层
        # embeds: [seq_len, batch_size, embedding_dim]
        lstm_out, _ = self.lstm(embeds)  # 经同由 LSTM 层
        # lstm_out: [seq_len, batch_size, hidden_dim]
        y = self.fc(lstm_out[-1])  # 取最后一步的输出进入最后的分类层
        # y: [batch_size, output_size]
        return y

In [38]:
# 定义全局变量
EMBEDDING_DIM = 128  # 词向量的大小为 128，embedding层输出维度
HIDDEN_DIM = 216  # 隐层大小为 216,LSTM层输出维度
VOCAB_DIM = 1000  # 词典大小为 1000，embedding_lookup词典大小
OUTPUT_SIZE = 3  # 输出类别为 3 类


## 3. 模型初始化

In [53]:
from torchsummary import summary
my_lstm = LSTM(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_DIM, OUTPUT_SIZE)
my_lstm

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

## 4. 测试

In [42]:
# seq_len 为 2，batch_size 为 4，各数字表示某单词的 id
x = torch.tensor([[1, 4, 5, 5], [3, 4, 9, 5]])
print(x.permute(1, 0).shape)
# x 大小为 [batch_size, seq_len],需要转置
y = my_lstm(x.permute(1, 0))# 每个batch输出一个分类概率向量
print(y) # y 大小为[seq_len, 3]
y.shape

torch.Size([4, 2])
tensor([[ 0.0347,  0.0891, -0.0613],
        [-0.0052,  0.0740, -0.0148]], grad_fn=<AddmmBackward>)


torch.Size([2, 3])